In [1]:
from freqtrade.arguments import Arguments
from freqtrade.configuration import Configuration
from freqtrade.analyze import Analyze
from freqtrade import constants

In [2]:
from argparse import Namespace

In [3]:
from importlib import reload

In [4]:
import plotly.offline as py
import plotly.graph_objs as go
#needed to have the display in the jupyter notebook:
py.init_notebook_mode(connected=True)

In [5]:
from freqtrade import exchange
import freqtrade.optimize as optimize
from freqtrade.optimize import backtesting


In [6]:
argList =  ['-s', 'BinHV45', 'backtesting', '--timerange=20180105-', '--ticker-interval', '5m', '--realistic-simulation']
arguments = Arguments(
        argList,
        'Fake command line for now '
    )
args = arguments.get_parsed_arg()

In [7]:
args.config = "../" + args.config
args.datadir = "../" + args.datadir

In [8]:
configuration = Configuration(args)
config = configuration.get_config()

2018-05-19 22:14:18,820 - freqtrade.configuration - INFO - Log level set to INFO
2018-05-19 22:14:18,820 - freqtrade.configuration - INFO - Parameter -i/--ticker-interval detected ...
2018-05-19 22:14:18,836 - freqtrade.configuration - INFO - Using ticker_interval: 5m ...
2018-05-19 22:14:18,836 - freqtrade.configuration - INFO - Parameter --realistic-simulation detected ...
2018-05-19 22:14:18,836 - freqtrade.configuration - INFO - Using max_open_trades: 3 ...
2018-05-19 22:14:18,836 - freqtrade.configuration - INFO - Parameter --timerange detected: 20180105- ...
2018-05-19 22:14:18,836 - freqtrade.configuration - INFO - Parameter --datadir detected: ../freqtrade\tests\testdata ...


In [9]:
exchange.init(config)
analyze = Analyze(config)

2018-05-19 22:14:19,015 - freqtrade.exchange - INFO - Instance is running with dry_run enabled
2018-05-19 22:14:19,023 - freqtrade.exchange - INFO - Using Exchange "Binance"
2018-05-19 22:14:19,486 - freqtrade.strategy.resolver - INFO - Using resolved strategy BinHV45 from 'd:\work\finances\freqtrade\myfreqtrade\freqtrade\freqtrade\strategy\..\..\user_data\strategies'
2018-05-19 22:14:19,487 - freqtrade.strategy.resolver - INFO - Override strategy 'ticker_interval' with value in config file: 5m.


In [10]:
pairs = config['exchange']['pair_whitelist']
ticker_interval = config['ticker_interval']

In [11]:
timerange = Arguments.parse_timerange(config.get('timerange'))
data = optimize.load_data(
    config['datadir'],
    pairs=pairs,
    ticker_interval=ticker_interval,
    refresh_pairs=config.get('refresh_pairs', False),
    timerange=timerange
)

In [12]:
def CumulProfit( df ):
    #cumul Profit :
    curSum = 100
    df.loc[:, 'CumulPL'] = curSum
    columnIndex = df.columns.get_loc('CumulPL')
    for i in range(1, len(df)):
        curSum = curSum * ( 1.0 + df.iloc[i]['profit_percent'])
        df.iloc[i,columnIndex] = curSum

In [13]:
def GraphRoundTripList( roundTripList, curveNames=[] ):
    data = []
    index = 0
    line100 = go.Scatter( x= [ roundTripList[0].iloc[0, :]['sell_date'], roundTripList[0].iloc[-1, :]['sell_date']], 
                          y=[100.0,100.0],
                          name="",
                          line = dict( color = ('rgb(200, 200, 200)'), width = 4)
                        )
    data.append(line100)
    for rt in roundTripList:
        graphName = 'pair' + str(index)
        if index < len(curveNames):
            graphName = curveNames[index]            
        graph = go.Scatter( x = rt['sell_date'], y=rt['CumulPL'], name=graphName)
        index += 1
        data.append(graph)
    fig = go.Figure(data=data)
    py.iplot(fig)

In [14]:
sell_profit_only = config.get('experimental', {}).get('sell_profit_only', False)
use_sell_signal = config.get('experimental', {}).get('use_sell_signal', False)
max_open_trades = config['max_open_trades']

In [15]:
preprocessed = analyze.tickerdata_to_dataframe(data)

In [16]:
min_date, max_date = backtesting.Backtesting.get_timeframe(preprocessed)

In [17]:
def getCumulProfitForStrat( strat ):
    config['strategy'] = strat
    analyze = Analyze(config)
    preprocessed = analyze.tickerdata_to_dataframe(data)
    myBacktesting = backtesting.Backtesting(config)
    results = myBacktesting.backtest(
            {
                'stake_amount': config.get('stake_amount'),
                'processed': preprocessed,
                'max_open_trades': max_open_trades,
                'realistic': config.get('realistic_simulation', False),
                'sell_profit_only': sell_profit_only,
                'use_sell_signal': use_sell_signal,
                'record': config.get('export')
            }
        )
    sortedResults = results.sort_values('sell_date') 
    sortedResults = sortedResults.reset_index(drop=True)
    CumulProfit( sortedResults )
    return sortedResults

#strat_list = [ "BinHV45", "Quickie", "Strategy006", "ASDTSRockwellTrading", "AverageStrategy", "BinHV27", "ClucMay72018", "CofiBitStrategy", "EMASkipPump", "MACDStrategy", "ReinforcedQuickie", "Simple", "SmoothOperator"]
strat_list = [ "BinHV45", "Quickie", "Strategy006", "ASDTSRockwellTrading", "AverageStrategy", "BinHV27", "ClucMay72018", "CofiBitStrategy"]

In [18]:
strat_list = [ "Low_BB","BinHV27", "BinHV45", "Quickie", "Strategy006", "ASDTSRockwellTrading", "AverageStrategy", "ClucMay72018", "CofiBitStrategy", "EMASkipPump", "MACDStrategy","ReinforcedQuickie", "Simple"]

In [ ]:
result_list = []
for strat in strat_list:
    stratResult = getCumulProfitForStrat( strat )
    result_list.append( stratResult )
GraphRoundTripList( result_list, strat_list)

2018-05-19 22:14:33,895 - freqtrade.strategy.resolver - INFO - Using resolved strategy Low_BB from 'd:\work\finances\freqtrade\myfreqtrade\freqtrade\freqtrade\strategy\..\..\user_data\strategies'
2018-05-19 22:14:33,895 - freqtrade.strategy.resolver - INFO - Override strategy 'ticker_interval' with value in config file: 5m.
2018-05-19 22:14:34,935 - freqtrade.strategy.resolver - INFO - Using resolved strategy Low_BB from 'd:\work\finances\freqtrade\myfreqtrade\freqtrade\freqtrade\strategy\..\..\user_data\strategies'
2018-05-19 22:14:34,935 - freqtrade.strategy.resolver - INFO - Override strategy 'ticker_interval' with value in config file: 5m.
2018-05-19 22:14:34,935 - freqtrade.exchange - INFO - Instance is running with dry_run enabled
2018-05-19 22:14:34,942 - freqtrade.exchange - INFO - Using Exchange "Binance"
2018-05-19 22:14:38,273 - freqtrade.strategy.resolver - INFO - Using resolved strategy BinHV27 from 'd:\work\finances\freqtrade\myfreqtrade\freqtrade\freqtrade\strategy\..\..